# A "Magic" Trick

*Hey kids, wanna see a function work and not work at the same time?*

## Set up

I was working with some data that only had names and information, but no ids. My job was to create that id. The data was ■■■■■■■■■■■■■■■■■■

Wait, looks like it's censored. Let me try again.

The data resembled some [magic cards](LINKTOMTGWIKI) that I downloaded from the internet. The ids mysteriously went missing, so my first job was to create them. Here's what the data looked like:

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
df = pd.read_csv('silver_bordered_set_cards.csv')
df.head()

,name,set_name,border_color,mana_cost,type_line,oracle_text,power,toughness,loyalty,collector_number,watermark,usd,eur
0,Goblin Bowling Team,Unglued,silver,{3}{R},Creature — Goblin,If Goblin Bowling Team would deal damage to a ...,1,1,NaN,44,NaN,0.20,0.35
1,Mise,Unhinged,silver,{U},Instant,"Choose a nonland card name, then reveal the to...",NaN,NaN,NaN,38,NaN,0.23,0.65
2,Steel Squirrel,Unstable,silver,{2},Artifact Creature — Squirrel,"Whenever you roll a 5 or higher on a die, Stee...",1,1,NaN,162,NaN,0.30,0.75
3,Knight of the Widget,Unstable,silver,{2}{W},Artifact Creature — Cyborg Knight,Vigilance\nKnight of the Widget's power and to...,*,*,NaN,13,orderofthewidget,0.10,0.19
4,Eye to Eye,Unhinged,silver,{2}{B},Instant,You and target creature's controller have a st...,NaN,NaN,NaN,54,NaN,0.28,0.48


Each card had a name, mana cost, card type, and some (oracle) text. Each of them came from a set of cards, which may include some cards that have been printed before. I knew this collection of cards came from different sets, and reprints would occur in them.

Luckily, I knew cards with the same name are the same, even across different sets. To showcase my point,

In [2]:
print(df['name'].duplicated().any())
df.loc[df['name'].duplicated(keep=False), ['name','set_name']].sort_values(by=['name','set_name'])

True


,name,set_name
609,AWOL,Unhinged
701,AWOL,Unhinged
417,AWOL,Unsanctioned
624,"Acornelia, Fashionable Filcher",The List (Unfinity Foil Edition)
650,"Acornelia, Fashionable Filcher",Unsanctioned
...,...,...
11,X,The List (Unfinity Foil Edition)
683,X,Unstable
96,Yet Another Aether Vortex,The List (Unfinity Foil Edition)
636,Yet Another Aether Vortex,Unhinged


Picking the first name as an example, we can see the it's the same card by checking the mana cost, type line, and oracle text:

(we could do more, but I'm too lazy to type them all out)

In [3]:
df.loc[df['name']=='AWOL',["oracle_text", "mana_cost", "type_line"]].drop_duplicates()

,oracle_text,mana_cost,type_line
417,Exile target attacking creature. Then remove i...,{2}{W},Instant


So I can just use the name to generate the id (or just use the name itself), and call it a day. Right?

## The Catch

When I said the same card name means the same card anywhere, that's mostly true. It's true for black bordered cards, which are more "serious" and fit the game more. As you may have spotted earlier, some of the cards don't have a black border -- they are silver-bordered! These silver-bordered sets are more comedic in tone, take themselves less seriously, and are more wild in both flavour and mechanics.

This also means some cards with the same name have different text.

In [4]:
df.loc[df["name"]=="Garbage Elemental", ["name", "set_name", "oracle_text"]].drop_duplicates()

,name,set_name,oracle_text
114,Garbage Elemental,Unstable,Frenzy 2 (Whenever this creature attacks and i...
370,Garbage Elemental,Unstable,"When Garbage Elemental enters the battlefield,..."
414,Garbage Elemental,Unstable,"Cascade (When you cast this spell, exile cards..."
421,Garbage Elemental,Unstable,"Battle cry (Whenever this creature attacks, ea..."
605,Garbage Elemental,Unstable,Last strike (This creature deals combat damage...
665,Garbage Elemental,Unstable,Unleash (You may have this creature enter the ...


In fact, a good number of cards across the sets behave like above.

In [5]:
unique_text_df = df.groupby(["name","set_name"], as_index=False, dropna=False)["oracle_text"].nunique()
unique_text_df[unique_text_df["oracle_text"]>1]

,name,set_name,oracle_text
36,B.F.M. (Big Furry Monster),Unglued,2
165,Everythingamajig,The List (Unfinity Foil Edition),2
166,Everythingamajig,Unstable,6
201,Garbage Elemental,Unstable,6
271,Ineffable Blessing,The List (Unfinity Foil Edition),2
272,Ineffable Blessing,Unstable,6
312,Knight of the Kitchen Sink,Unstable,6
478,Sly Spy,Unstable,6
588,Very Cryptic Command,Unstable,6


Let's save them to a new column so we can use it for later.

In [6]:
df["has_variation"] = df["name"].isin(unique_text_df.loc[unique_text_df["oracle_text"]>1, "name"])
original_df = df.copy() # don't mind me, just saving a copy in case of issues down the line

## The Function

Now we've found the catch, let's tackle the original problem of assigning ids. We can decide on the id function later, for now let's just put them into tuples.
<!-- I know there are some number of issues with this approach already, but bear with me -->

For those without variations, we only the name is enough. For those with variations, we have to include the oracle text as well. So here's what the keys/ids would look

In [7]:
np.where(
    df["has_variation"],
    df[["has_variation","name","oracle_text"]].apply(tuple, axis=1),
    df[["has_variation","name"]].apply(tuple, axis=1),
)

array([(False, 'Goblin Bowling Team'), (False, 'Mise'),
       (False, 'Steel Squirrel'), (False, 'Knight of the Widget'),
       (False, 'Eye to Eye'), (False, 'Split Screen'),
       (False, 'Six-y Beast'), (False, 'Handy Dandy Clone Machine'),
       (False, 'Aesthetic Consultation'), (False, 'Togglodyte'),
       (False, 'Organ Harvest'), (False, 'X'),
       (False, 'Cardboard Carapace'), (False, 'Novellamental'),
       (False, 'Moniker Mage'), (False, 'The Grand Calcutron'),
       (False, 'GO TO JAIL'), (False, 'Insufferable Syphon'),
       (False, 'Humming-'), (False, 'Faerie Aerie'),
       (False, 'Do-It-Yourself Seraph'), (False, 'Monkey Monkey Monkey'),
       (False, 'Handcuffs'), (False, 'Richard Garfield, Ph.D.'),
       (False, 'Smart Ass'), (False, 'Rare-B-Gone'),
       (False, 'Split Screen'), (False, "Ass Whuppin'"),
       (False, 'Contraption Cannon'), (False, 'Booster Tutor'),
       (False, 'Old Fogey'), (False, 'Standing Army'),
       (False, 'Goblin Haberda

Great! Let's chuck it in a function. I want to make the list of columns changable in different parts of the code, so let's put them into parameters. We also want to add typing for readability as well, and I don't want to use np.array as a type, so I've put the result into a pandas series.

In [8]:
def gen_key(df, no_variation_cols: list[str], have_variation_cols: list[str]) -> pd.Series:
    if "has_variation" not in no_variation_cols:
        no_variation_cols = ["has_variation"] + no_variation_cols
    if "has_variation" not in have_variation_cols:
        have_variation_cols = ["has_variation"] + have_variation_cols
    return pd.Series(np.where(
        df["has_variation"],
        df[have_variation_cols].apply(tuple, axis=1),
        df[no_variation_cols].apply(tuple, axis=1)
    ))

Wait, I just got word from my super that we're only worried about cards whose type line starts with "Artifact". No worries, let's add a filter and then run our newly constructed function:

In [9]:
artifact_df = df[df["type_line"].str.startswith("Artifact")]
artifact_df["key"] = gen_key(artifact_df, ["name"], ["name", "oracle_text"])
artifact_df[["name","set_name","oracle_text","key"]].head(10)

,name,set_name,oracle_text,key
2,Steel Squirrel,Unstable,"Whenever you roll a 5 or higher on a die, Stee...","(False, Split Screen)"
3,Knight of the Widget,Unstable,Vigilance\nKnight of the Widget's power and to...,"(False, Handy Dandy Clone Machine)"
5,Split Screen,Unstable,"When Split Screen enters the battlefield, shuf...","(False, Insufferable Syphon)"
7,Handy Dandy Clone Machine,The List (Unfinity Foil Edition),"{2}, {T}: Create a 2/2 colorless Homunculus cr...","(False, Do-It-Yourself Seraph)"
9,Togglodyte,Unhinged,Togglodyte enters the battlefield turned on.\n...,"(False, Contraption Cannon)"
17,Insufferable Syphon,Unstable,"Whenever you crank Insufferable Syphon, target...","(False, Blurry Beeble)"
19,Faerie Aerie,Unstable,"Whenever you crank Faerie Aerie, create two 1/...","(False, Dogsnail Engine)"
20,Do-It-Yourself Seraph,The List (Unfinity Foil Edition),Flying\nWhenever Do-It-Yourself Seraph attacks...,"(False, Enraged Killbot)"
26,Split Screen,The List (Unfinity Foil Edition),"When Split Screen enters the battlefield, shuf...","(False, Auto-Key)"
28,Contraption Cannon,Unstable,"{2}, Sacrifice Contraption Cannon: It deals da...","(False, Water Gun Balloon Game)"


Well, that doesn't look correct. Let me try again, but without the "Artifact" requirement:

In [10]:
df["key2"] = gen_key(df, ["name"], ["name", "oracle_text"])
display(df[["name","set_name","oracle_text","key2"]].head())
display(df.loc[df["name"]=="Garbage Elemental",["name","set_name","oracle_text","key2"]])

,name,set_name,oracle_text,key2
0,Goblin Bowling Team,Unglued,If Goblin Bowling Team would deal damage to a ...,"(False, Goblin Bowling Team)"
1,Mise,Unhinged,"Choose a nonland card name, then reveal the to...","(False, Mise)"
2,Steel Squirrel,Unstable,"Whenever you roll a 5 or higher on a die, Stee...","(False, Steel Squirrel)"
3,Knight of the Widget,Unstable,Vigilance\nKnight of the Widget's power and to...,"(False, Knight of the Widget)"
4,Eye to Eye,Unhinged,You and target creature's controller have a st...,"(False, Eye to Eye)"


,name,set_name,oracle_text,key2
114,Garbage Elemental,Unstable,Frenzy 2 (Whenever this creature attacks and i...,"(True, Garbage Elemental, Frenzy 2 (Whenever t..."
370,Garbage Elemental,Unstable,"When Garbage Elemental enters the battlefield,...","(True, Garbage Elemental, When Garbage Element..."
414,Garbage Elemental,Unstable,"Cascade (When you cast this spell, exile cards...","(True, Garbage Elemental, Cascade (When you ca..."
421,Garbage Elemental,Unstable,"Battle cry (Whenever this creature attacks, ea...","(True, Garbage Elemental, Battle cry (Whenever..."
605,Garbage Elemental,Unstable,Last strike (This creature deals combat damage...,"(True, Garbage Elemental, Last strike (This cr..."
665,Garbage Elemental,Unstable,Unleash (You may have this creature enter the ...,"(True, Garbage Elemental, Unleash (You may hav..."


Wait, what?

In the actual code and data I was working on, all the above was buried inside 10 other different files and a lot of more transformations. It took me almost a day to narrow it down to this function, and a lot longer to figure out the issue with the above. I tried writing tests for it, all of which passed. I double checked the data, which looked to be fine. This little utility function was used in multiple places across files, and rewriting all of them would take significant time.

I was just about to give up when I finally identified the issue.

## The reveal

Nothing's wrong with values, the problem was with the index:

In [11]:
gen_key(artifact_df, ["name"], ["name", "oracle_text"])

0                                (False, Steel Squirrel)
1                          (False, Knight of the Widget)
2                                  (False, Split Screen)
3                     (False, Handy Dandy Clone Machine)
4                                    (False, Togglodyte)
                             ...                        
140    (True, Knight of the Kitchen Sink, First strik...
141                          (False, World-Bottling Kit)
142                                  (False, Mana Screw)
143                                  (False, Sap Sucker)
144                              (False, Jamming Device)
Length: 145, dtype: object

Compare this to the index of the `artifact_df`:

In [12]:
artifact_df.index

Index([  2,   3,   5,   7,   9,  17,  19,  20,  26,  28,
       ...
       668, 671, 681, 685, 691, 694, 702, 703, 704, 706],
      dtype='int64', length=145)

They don't align!

This is why the key for "Steel Squrriel" was `(False, "Split Screen")`. The card's index in the filtered dataframe was 2, but in the series it had an index of 0. At the same time, the third key in the series had an index of 2, so that's used instead.

This also means that the some elements in `artifact_df` supposedly won't have any keys, but now we know it's because its index doesn't appear in the series. We can even pinpoint to say it's all rows with `index >= len(artifact_df)`.

(I subtracted 10 to show the elements just before it)

In [13]:
artifact_df.loc[artifact_df.index >= len(artifact_df)-10, ["name","set_name","oracle_text","key"]]

,name,set_name,oracle_text,key
141,Handy Dandy Clone Machine,Unstable,"{2}, {T}: Create a 2/2 colorless Homunculus cr...","(False, World-Bottling Kit)"
144,Time Machine,Unhinged,{T}: Exile Time Machine and target nontoken cr...,"(False, Jamming Device)"
145,Knight of the Kitchen Sink,Unstable,"First strike, protection from black borders (N...",NaN
149,Riveting Rigger,Unstable,"When Riveting Rigger enters the battlefield, y...",NaN
152,Knight of the Kitchen Sink,Unstable,"First strike, protection from watermarks (Noth...",NaN
...,...,...,...,...
694,Knight of the Kitchen Sink,Unstable,"First strike, protection from even collector n...",NaN
702,World-Bottling Kit,Unhinged,"{5}, Sacrifice World-Bottling Kit: Choose a Ma...",NaN
703,Mana Screw,Unhinged,"{1}: Flip a coin. If you win the flip, add {C}...",NaN
704,Sap Sucker,Unstable,"Whenever you crank Sap Sucker, add {G}. Until ...",NaN


Now we've understood what went wrong, let's fix it. The easiest fix is to not wrap the result in a pandas series -- no index, no problems!

In [14]:
def gen_key_fixed_np(df, no_variation_cols: list[str], have_variation_cols: list[str]) -> np.array:
    if "has_variation" not in no_variation_cols:
        no_variation_cols = ["has_variation"] + no_variation_cols
    if "has_variation" not in have_variation_cols:
        have_variation_cols = ["has_variation"] + have_variation_cols
    return np.where(
        df["has_variation"],
        df[have_variation_cols].apply(tuple, axis=1),
        df[no_variation_cols].apply(tuple, axis=1)
    )

In [15]:
artifact_df = df[df["type_line"].str.startswith("Artifact")]
artifact_df["key3"] = gen_key_fixed_np(artifact_df, ["name"], ["name", "oracle_text"])
artifact_df[["name","set_name","oracle_text","key3"]].head(10)

,name,set_name,oracle_text,key3
2,Steel Squirrel,Unstable,"Whenever you roll a 5 or higher on a die, Stee...","(False, Steel Squirrel)"
3,Knight of the Widget,Unstable,Vigilance\nKnight of the Widget's power and to...,"(False, Knight of the Widget)"
5,Split Screen,Unstable,"When Split Screen enters the battlefield, shuf...","(False, Split Screen)"
7,Handy Dandy Clone Machine,The List (Unfinity Foil Edition),"{2}, {T}: Create a 2/2 colorless Homunculus cr...","(False, Handy Dandy Clone Machine)"
9,Togglodyte,Unhinged,Togglodyte enters the battlefield turned on.\n...,"(False, Togglodyte)"
17,Insufferable Syphon,Unstable,"Whenever you crank Insufferable Syphon, target...","(False, Insufferable Syphon)"
19,Faerie Aerie,Unstable,"Whenever you crank Faerie Aerie, create two 1/...","(False, Faerie Aerie)"
20,Do-It-Yourself Seraph,The List (Unfinity Foil Edition),Flying\nWhenever Do-It-Yourself Seraph attacks...,"(False, Do-It-Yourself Seraph)"
26,Split Screen,The List (Unfinity Foil Edition),"When Split Screen enters the battlefield, shuf...","(False, Split Screen)"
28,Contraption Cannon,Unstable,"{2}, Sacrifice Contraption Cannon: It deals da...","(False, Contraption Cannon)"


Alternatively, if we wanted to have it return a pandas series, we can suppliment it with the index of the dataframe.

In [16]:
def gen_key_fixed_pd(df, no_variation_cols: list[str], have_variation_cols: list[str]) -> pd.Series:
    if "has_variation" not in no_variation_cols:
        no_variation_cols = ["has_variation"] + no_variation_cols
    if "has_variation" not in have_variation_cols:
        have_variation_cols = ["has_variation"] + have_variation_cols
    return pd.Series(
        np.where(
            df["has_variation"],
            df[have_variation_cols].apply(tuple, axis=1),
            df[no_variation_cols].apply(tuple, axis=1)
        ),
        index = df.index
    )

In [17]:
artifact_df = df[df["type_line"].str.startswith("Artifact")]
artifact_df["key4"] = gen_key_fixed_pd(artifact_df, ["name"], ["name", "oracle_text"])
artifact_df[["name","set_name","oracle_text","key4"]].head(10)

,name,set_name,oracle_text,key4
2,Steel Squirrel,Unstable,"Whenever you roll a 5 or higher on a die, Stee...","(False, Steel Squirrel)"
3,Knight of the Widget,Unstable,Vigilance\nKnight of the Widget's power and to...,"(False, Knight of the Widget)"
5,Split Screen,Unstable,"When Split Screen enters the battlefield, shuf...","(False, Split Screen)"
7,Handy Dandy Clone Machine,The List (Unfinity Foil Edition),"{2}, {T}: Create a 2/2 colorless Homunculus cr...","(False, Handy Dandy Clone Machine)"
9,Togglodyte,Unhinged,Togglodyte enters the battlefield turned on.\n...,"(False, Togglodyte)"
17,Insufferable Syphon,Unstable,"Whenever you crank Insufferable Syphon, target...","(False, Insufferable Syphon)"
19,Faerie Aerie,Unstable,"Whenever you crank Faerie Aerie, create two 1/...","(False, Faerie Aerie)"
20,Do-It-Yourself Seraph,The List (Unfinity Foil Edition),Flying\nWhenever Do-It-Yourself Seraph attacks...,"(False, Do-It-Yourself Seraph)"
26,Split Screen,The List (Unfinity Foil Edition),"When Split Screen enters the battlefield, shuf...","(False, Split Screen)"
28,Contraption Cannon,Unstable,"{2}, Sacrifice Contraption Cannon: It deals da...","(False, Contraption Cannon)"


Phew! We've finally fixed the problem. Now we can get back on track and focus on whatever we were on before.


## The takeaways

*What have we learned today, kids?*

*Don't use `np.where`!*

Well, not quite.

Do use it when you're dealing with numpy arrays, but with pandas, you have to be a bit more mindful about the indexes, especially with dataframes being filtered and transformed everywhere. If you don't remember whether a dataframe has been filtered or it's a fresh copy, better make the function not aware of its index, or just reset it if you are cautious.

Oh, and don't use apply like I did. Find another way to do it:

In [18]:
def gen_key_no_apply(df, no_variation_cols: list[str], have_variation_cols: list[str]) -> pd.Series:
    if "has_variation" not in no_variation_cols:
        no_variation_cols = ["has_variation"] + no_variation_cols
    if "has_variation" not in have_variation_cols:
        have_variation_cols = ["has_variation"] + have_variation_cols
    return pd.Series(
        np.where(
            df["has_variation"],
            df.set_index(have_variation_cols).index,
            df.set_index(no_variation_cols).index,
        ),
        index = df.index
    )
artifact_df = df[df["type_line"].str.startswith("Artifact")]
artifact_df["key5"] = gen_key_no_apply(artifact_df, ["name"], ["name", "oracle_text"])
artifact_df[["name","set_name","oracle_text","key5"]].head(10)

,name,set_name,oracle_text,key5
2,Steel Squirrel,Unstable,"Whenever you roll a 5 or higher on a die, Stee...","(False, Steel Squirrel)"
3,Knight of the Widget,Unstable,Vigilance\nKnight of the Widget's power and to...,"(False, Knight of the Widget)"
5,Split Screen,Unstable,"When Split Screen enters the battlefield, shuf...","(False, Split Screen)"
7,Handy Dandy Clone Machine,The List (Unfinity Foil Edition),"{2}, {T}: Create a 2/2 colorless Homunculus cr...","(False, Handy Dandy Clone Machine)"
9,Togglodyte,Unhinged,Togglodyte enters the battlefield turned on.\n...,"(False, Togglodyte)"
17,Insufferable Syphon,Unstable,"Whenever you crank Insufferable Syphon, target...","(False, Insufferable Syphon)"
19,Faerie Aerie,Unstable,"Whenever you crank Faerie Aerie, create two 1/...","(False, Faerie Aerie)"
20,Do-It-Yourself Seraph,The List (Unfinity Foil Edition),Flying\nWhenever Do-It-Yourself Seraph attacks...,"(False, Do-It-Yourself Seraph)"
26,Split Screen,The List (Unfinity Foil Edition),"When Split Screen enters the battlefield, shuf...","(False, Split Screen)"
28,Contraption Cannon,Unstable,"{2}, Sacrifice Contraption Cannon: It deals da...","(False, Contraption Cannon)"
